In [1]:
from td_api import Account
from datetime import datetime, timedelta

acc = Account("keys.json")


symbol = "AMD"
days = 30
strike_count = 50

from_date = datetime.now()
to_date = from_date + timedelta(days=days)
data = acc.get_options_chain(symbol, from_date, to_date, strike_count=strike_count)


In [6]:
import numpy as np
import pandas as pd

df = {}

for i in range(3):
    sub_df = {}
    for j in range(10):
        sub_df["sub_%d" % j] = np.random.random(20)
    df["df_%d" % i] = sub_df

df = pd.DataFrame(df)
print(df)

Index(['sub_0', 'sub_1', 'sub_2', 'sub_3', 'sub_4', 'sub_5', 'sub_6', 'sub_7',
       'sub_8', 'sub_9'],
      dtype='object')
